In [3]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, urlparse, parse_qs

def download_one_episode(title, no, url):
    """
    하나의 웹툰 회차 이미지를 다운로드하는 함수.
    (try-except 없음 - 오류 발생 시 프로그램 종료)
    """
    print(f"\n--- 웹툰 다운로드 시작 ---")
    print(f"웹툰 제목: {title}, 회차 번호: {no}, URL: {url}")

    req_header = {
        'Referer': url,
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }

    try:
        res = requests.get(url, headers=req_header)
        res.raise_for_status() # HTTP 오류가 발생하면 예외를 발생시킵니다.
    except requests.exceptions.RequestException as e:
        print(f"오류: {url} 요청 중 오류 발생: {e}")
        return

    soup = BeautifulSoup(res.text, 'html.parser')

    viewer_div = soup.select_one('div.wt_viewer')
    if not viewer_div:
        print(f"오류: '{url}'에서 wt_viewer div를 찾을 수 없습니다. (뷰어 클래스명 변경 가능성)")
        return

    imgurl_list = []
    for img_tag in viewer_div.find_all('img'):
        src = img_tag.get('src')
        data_src = img_tag.get('data-src')

        if src and src.startswith('https://image-comic.pstatic.net/webtoon/'):
            imgurl_list.append(src)
        elif data_src and data_src.startswith('https://image-comic.pstatic.net/webtoon/'):
            imgurl_list.append(data_src)
        
    if not imgurl_list:
        print(f"오류: '{url}'에서 이미지 URL을 찾을 수 없습니다. (이미지 URL 패턴 변경 가능성)")
        return

    print(f"총 {len(imgurl_list)}개의 이미지를 찾았습니다.")

    # 파일 시스템에 안전한 제목으로 변환 (특수문자 제거)
    safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '.', '_', '-')).strip()
    dir_name = os.path.join('img', safe_title, str(no))
    
    print(f"이미지 저장 디렉토리: {dir_name}")

    os.makedirs(dir_name, exist_ok=True) # exist_ok=True를 사용하여 이미 존재하는 경우 오류 방지

    for idx, img_url in enumerate(imgurl_list, 1):
        full_img_url = urljoin(url, img_url)

        try:
            img_res = requests.get(full_img_url, headers=req_header, stream=True) 
            img_res.raise_for_status() # 이미지 다운로드 중 오류 발생 시 예외 발생
        except requests.exceptions.RequestException as e:
            print(f"경고: {full_img_url} 이미지 다운로드 실패: {e}")
            continue

        file_name = os.path.basename(full_img_url)
        file_path = os.path.join(dir_name, file_name)

        with open(file_path, 'wb') as file:
            for chunk in img_res.iter_content(chunk_size=8192):
                file.write(chunk)
        
        print(f'{idx}. {file_path} (다운로드 완료)')
    
    print(f"--- '{title}' {no}회차 다운로드 완료 ---\n")


def download_all_episode(title, list_page_url):
    """
    하나의 웹툰의 목록 페이지에서 여러 회차의 이미지를 다운로드하는 함수.
    (try-except 없음 - 오류 발생 시 프로그램 종료)
    """
    print(f"\n--- '{title}' 웹툰의 모든 회차 다운로드 시작 ---")
    print(f"목록 페이지 URL: {list_page_url}")

    req_header = {
        'Referer': list_page_url,
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }

    try:
        res = requests.get(list_page_url, headers=req_header)
        res.raise_for_status() # HTTP 오류가 발생하면 예외를 발생시킵니다.
    except requests.exceptions.RequestException as e:
        print(f"오류: 목록 페이지 접근 실패: {e}")
        return

    soup = BeautifulSoup(res.text, 'html.parser')
    
    # 스크린샷을 기반으로 업데이트된 CSS 선택자
    # ul 태그의 클래스 이름을 정확히 지정합니다.
    # 만약 이 코드가 나중에 작동하지 않는다면, EpisodeList__episode_list-- 뒤의 문자열(_N3KS)이 변경되었을 가능성이 큽니다.
    episode_links = soup.select('ul.EpisodeList__episode_list--_N3KS li a[href*="/webtoon/detail"]')
    
    if not episode_links:
        print(f"오류: 목록 페이지에서 회차 링크를 찾을 수 없습니다. CSS 셀렉터를 확인해주세요. (현재 시도: 'ul.EpisodeList__episode_list--_N3KS li a[href*=\"/webtoon/detail\"]')")
        return

    print(f"총 {len(episode_links)}개의 회차 링크를 찾았습니다.")

    # 요청에 따라 최대 20개의 회차를 다운로드합니다.
    episodes_to_download_count = min(len(episode_links), 20) 
    
    print(f"상위 {episodes_to_download_count}개 회차 다운로드를 시도합니다.")

    for i in range(episodes_to_download_count):
        link_tag = episode_links[i]
        relative_url = link_tag.get('href') 
        
        if not relative_url:
            print(f"경고: 유효하지 않은 링크를 건너뜁니다.")
            continue

        full_episode_url = urljoin(list_page_url, relative_url)
        
        parsed_url = urlparse(full_episode_url)
        query_params = parse_qs(parsed_url.query)
        episode_no = query_params.get('no', [None])[0] 

        if not episode_no:
            print(f"경고: 회차 번호를 찾을 수 없어 '{full_episode_url}'를 건너뜁니다.")
            continue

        print(f"\n--- 회차 {episode_no} 다운로드 시작 ---")
        download_one_episode(title, episode_no, full_episode_url)
    
    print(f"\n--- '{title}' 웹툰의 모든 회차 다운로드 완료 ---")


# --- 실행 예시 ---
webtoon_title = '배달왕'
webtoon_list_url = 'https://comic.naver.com/webtoon/list?titleId=823933'

# download_all_episode 함수 호출
download_all_episode(webtoon_title, webtoon_list_url)


--- '배달왕' 웹툰의 모든 회차 다운로드 시작 ---
목록 페이지 URL: https://comic.naver.com/webtoon/list?titleId=823933
오류: 목록 페이지에서 회차 링크를 찾을 수 없습니다. CSS 셀렉터를 확인해주세요. (현재 시도: 'ul.EpisodeList__episode_list--_N3KS li a[href*="/webtoon/detail"]')
